<a href="https://colab.research.google.com/github/NgPcAnhh/Data/blob/master/nckh_crawldata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 23.4 MB/s eta 0:00:00


In [2]:
import requests
import os
import pandas as pd
from bs4 import BeautifulSoup
import re
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# mã cổ phiếu cần theo dõi tin tức
cp = 'VIC'

Crawl data cho CafeF

In [7]:
search_url = 'https://cafef.vn'
save_path = '/content/drive/MyDrive/NCKH-2025/data_raw_cafef/data_raw_cafef.xlsx'

# Function to save article content to a DataFrame
def save_article(data, item, session):
    article_url = search_url + item['href']
    print(f"Fetching article: {article_url}")  # Debug: Print URL being fetched
    response = session.get(article_url)
    print(f"Status code: {response.status_code}")  # Debug: Print status code
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        detail_content = soup.find('div', class_='detail-content afcbc-body')
        if detail_content:
            paragraphs = detail_content.find_all('p')
            content = "\n\n".join(paragraph.get_text(strip=True) for paragraph in paragraphs if paragraph.get_text(strip=True))
            sapo_tag = soup.find('h2', class_='sapo')
            sapo = sapo_tag.get_text(strip=True) if sapo_tag else ''
            category_tag = soup.find('a', class_='category-page__name cat')
            category = category_tag.get_text(strip=True) if category_tag else ''
            author_tag = soup.find('div', class_='t-contentdetail content_source').find('p', class_='author')
            author = author_tag.get_text(strip=True) if author_tag else ''
            if content:
                data.append({
                    'id': item['time_title'].replace('-', '').replace(':', '').replace(' ', '') + sapo,
                    'title': item['docnhanh_title'],
                    'sapo': sapo,
                    'content': content,
                    'date': item['time_title'],
                    'url': article_url,
                    'category': category,
                    'author': author
                })
                print(f"Article saved: {item['docnhanh_title']}")
            else:
                print(f"No paragraphs found in article: {article_url}")
        else:
            print(f"No detail content found in article: {article_url}")
    else:
        print(f"Failed to retrieve article: {article_url}")

# Function to crawl a single page
def crawl_page(url, session, data):
    response = session.get(url)
    print(f"Response status code: {response.status_code}")
    soup = BeautifulSoup(response.content, 'html.parser')

    # Tìm thẻ div có id là divEvents
    div_events = soup.find('div', id='divEvents')
    print(f"div_events found: {div_events is not None}")

    # Tìm tất cả các thẻ <li> trong thẻ divEvents
    events = div_events.find_all('li')
    print(f"Number of events found: {len(events)}")

    for event in events:
        time_title = event.find('span', class_='timeTitle').get_text(strip=True)
        docnhanh_title_tag = event.find('a', class_='docnhanhTitle')
        docnhanh_title = docnhanh_title_tag['title']
        href = docnhanh_title_tag['href']
        print(f"Event found: {time_title} - {docnhanh_title}")

        item = {
            'time_title': time_title,
            'docnhanh_title': docnhanh_title,
            'href': href
        }

        save_article(data, item, session)

    # Tạo URL cho trang tiếp theo sử dụng đúng AJAX endpoint
    page_index = url.split('PageIndex=')[1].split('&')[0] if 'PageIndex=' in url else '1'
    next_page = int(page_index) + 1
    next_url = f"https://cafef.vn/du-lieu/Ajax/Events_RelatedNews_New.aspx?symbol={cp}&floorID=0&configID=0&PageIndex={next_page}&PageSize=30&Type=2"

    return next_url

# Main function to crawl data
def crawl_data(base_url, num_pages):
    session = requests.Session()
    data = []
    current_url = base_url
    for page in range(1, num_pages + 1):
        print(f"\nCrawling page {page}")
        current_url = crawl_page(current_url, session, data)
        if not current_url:
            print("No more pages to crawl")
            break
        # Thêm delay giữa các request để tránh quá tải server
        time.sleep(2)

    # Save data to Excel file
    df = pd.DataFrame(data)
    df.to_excel(save_path, index=False)
    print(f"Data saved to {save_path}")

# Start crawling
num_pages = 5  # Số lần lặp lại quá trình crawl
base_url = f"https://cafef.vn/du-lieu/Ajax/Events_RelatedNews_New.aspx?symbol={cp}&floorID=0&configID=0&PageIndex=1&PageSize=30&Type=2"
crawl_data(base_url, num_pages)



Crawling page 1
Response status code: 200
div_events found: True
Number of events found: 30
Event found: 15/01/2025 14:11 - Tỷ phú Phạm Nhật Vượng đã đầu tư những dự án nào tại Thanh Hóa?
Fetching article: https://cafef.vn/du-lieu/vic-2115508/ty-phu-pham-nhat-vuong-da-dau-tu-nhung-du-an-nao-tai-thanh-hoa.chn
Status code: 200
Article saved: Tỷ phú Phạm Nhật Vượng đã đầu tư những dự án nào tại Thanh Hóa?
Event found: 15/01/2025 00:00 - VIC: 24.1.2025, ngày GDKHQ lấy ý kiến cổ đông bằng văn bản
Fetching article: https://cafef.vn/du-lieu/vic-2116094/vic-2412025-ngay-gdkhq-lay-y-kien-co-dong-bang-van-ban.chn
Status code: 200
No detail content found in article: https://cafef.vn/du-lieu/vic-2116094/vic-2412025-ngay-gdkhq-lay-y-kien-co-dong-bang-van-ban.chn
Event found: 14/01/2025 00:00 - VIC: Thông báo ngày ĐKCC để thực hiện quyền tham gia lấy ý kiến cổ đông bằng văn bản
Fetching article: https://cafef.vn/du-lieu/vic-2115277/vic-thong-bao-ngay-dkcc-de-thuc-hien-quyen-tham-gia-lay-y-kien-co-d

Crawl data cho vn economy

In [5]:
import requests
import os
import pandas as pd
from bs4 import BeautifulSoup

# Setup
search_url = 'https://vneconomy.vn/'
save_path = '/content/drive/MyDrive/NCKH-2025/data_raw/data_raw_vneconomy.xlsx'

# Function to save article content to a DataFrame
def save_article(data, item, session):
    article_url = search_url + item['UrlArticle']
    print(f"Fetching article: {article_url}")  # Debug: Print URL being fetched
    response = session.get(article_url)
    print(f"Status code: {response.status_code}")  # Debug: Print status code
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        detail_content = soup.find('div', class_='detail__content')
        if detail_content:
            paragraphs = detail_content.find_all('p')
            content = "\n\n".join(paragraph.get_text(strip=True) for paragraph in paragraphs if paragraph.get_text(strip=True))
            if content:
                data.append({
                    'id': item['Id'],
                    'title': item['Title'],
                    'sapo': item['Sapo'],
                    'content': content,
                    'date': item['PublishedDate'],
                    'url': article_url,
                    'category': item['CatName'],
                    'author': item['NameAuthor']
                })
                print(f"Article saved: {item['Title']}")
            else:
                print(f"No paragraphs found in article: {article_url}")
        else:
            print(f"No detail content found in article: {article_url}")
    else:
        print(f"Failed to retrieve article: {article_url}")

# Main function to crawl data
def crawl_data(base_url, num_pages):
    session = requests.Session()
    data = []
    for page in range(1, num_pages + 1):
        url = base_url + str(page)
        print(f"Fetching page: {url}")  # Debug: Print URL being fetched
        response = session.get(url)
        print(f"Status code: {response.status_code}")  # Debug: Print status code
        if response.status_code == 200:
            try:
                json_data = response.json().get('List', [])
                for item in json_data:
                    print(f"Item: {item}")  # Debug: Print each item data
                    if all(key in item for key in ['Id', 'Title', 'PublishedDate', 'UrlArticle', 'CatName', 'Sapo', 'NameAuthor']):
                        save_article(data, item, session)
            except json.JSONDecodeError as e:
                print(f"Failed to decode JSON from {url}: {e}")
        else:
            print(f"Failed to retrieve data from: {url}")

    # Save data to Excel file
    df = pd.DataFrame(data)
    df.to_excel(save_path, index=False)
    print(f"Data saved to {save_path}")

# Start crawling
num_pages = 10  # Số lần lặp lại quá trình crawl
base_url = f'https://search.hemera.com.vn/search/1/VIC/time/'  # Thay đổi mã CP theo nhu cầu
crawl_data(base_url, num_pages)


Fetching page: https://search.hemera.com.vn/search/1/VIC/time/1
Status code: 200
Item: {'Id': '1_638696546671771044', 'Title': 'Dự án <em>VIC</em> Grand Square được vinh danh tại giải thưởng bất động sản Đông Nam Á', 'JsonTag': '[{"Name":"Dự án VIC Grand Square","Url":"du-an-vic-grand-square"},{"Name":"Vneconomy","Url":"vneconomy"}]', 'PublishedDate': '2024-12-13T03:12:58.703Z', 'CatName': 'Bất động sản', 'UrlCate': 'dia-oc', 'Sapo': ' sạn Four Seasons Hotel Bangkok, Thái Lan dự án <em>VIC</em> Grand Square của chủ đầu tư <em>VIC</em> Group đã bất ngờ', 'Avatar': '2024/12/13/z6125163269366-46a397203c4fa7d3fb94f6c2091f739c.jpg', 'UrlArticle': 'du-an-vic-grand-square-duoc-vinh-danh-tai-giai-thuong-bat-dong-san-dong-nam-a', 'NameAuthor': 'Khánh  Huyền'}
Fetching article: https://vneconomy.vn/du-an-vic-grand-square-duoc-vinh-danh-tai-giai-thuong-bat-dong-san-dong-nam-a
Status code: 200
Article saved: Dự án <em>VIC</em> Grand Square được vinh danh tại giải thưởng bất động sản Đông Nam Á
Ite